In [1]:
from main_agent import Main_agent
import requests
from PIL import Image
from io import BytesIO
import tempfile
import os
from spire.doc import Document,FileFormat
from spire.doc.common import *
import gradio as gr

In [2]:
agent=Main_agent()

In [3]:
def temp_image(url:str):
    image=requests.get(url)
    image=Image.open(BytesIO(image.content))
    with tempfile.NamedTemporaryFile(mode='w', suffix='.png', delete=False) as temp_file:
        image.save(temp_file.name)
        image_file_path = temp_file.name
    return image_file_path

In [4]:
def paper_to_markdown(paper:dict) -> str:
    """
    Convert a Paper_layout object into a markdown string.
    
    Args:
        paper (Paper_layout): The paper layout to convert
        
    Returns:
        str: The complete markdown document
    """
    
    markdown = []
    
    # Add title
    markdown.append("#" + paper.get('title'))
    markdown.append("\n\n")
    if paper.get('image_url'):
        markdown.append(f"![Image]({paper.get('image_url')})\n")
        markdown.append("\n\n")
    
    # Add body sections
    for section in paper.get('paragraphs'):
        markdown.append(f"## {section.get('title')}\n")
        if section.get('content'):
            markdown.append(section.get('content'))
            markdown.append("\n")


        markdown.append("\n\n")
    
    if paper.get('table'):
        markdown.append(paper.get('table'))
        markdown.append("\n")
    # Add references
    markdown.append("## References\n")
    markdown.append(str(paper.get('references')))
    
    return "".join(markdown)

In [5]:
def save_markdown_as_docx(markdown:str):
# Create a temporary file
    with tempfile.NamedTemporaryFile(mode='w', suffix='.md', delete=False) as temp_file:
        temp_file.write(markdown)
        temp_file_path = temp_file.name

    # Create a new document and load from the temporary files
    document = Document()
    document.LoadFromFile(temp_file_path)

    # Save as DOCX
    document.SaveToFile("output.docx", FileFormat.Docx)

    # Clean up the temporary file
    os.unlink(temp_file_path)

In [6]:


def reset():
    agent.reset()

research_paper=agent.deps.deep_search_results

def update_markdown():
    """Get the current research paper markdown"""
    if agent.deps.deep_search_results:
        research_paper = agent.deps.deep_search_results[-1]
        if research_paper:
            return paper_to_markdown(research_paper)
    return "No research paper available yet."






In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# Research Assistant Chatbot")
    
    with gr.Row():
        with gr.Column():
            chatbot = gr.Chatbot(type='messages')
            msg = gr.Textbox(label="Your Query")
            with gr.Row():
                submit_btn = gr.Button("Submit")
                reset_btn = gr.Button("Reset")
        
        with gr.Column():
            markdown_display = gr.Markdown(label="Research Paper")
            save_btn = gr.Button("Save as DOCX")
            save_status = gr.Textbox(label="Save Status", interactive=False)
    
    # Function to handle chat
    async def respond(input, history):
        history=[]
        response =await agent.chat(input)
        history.append({"role": "user", "content": input})
        history.append({"role": "assistant", "content": str(response)})
        return"", history, update_markdown()
    
    # Function to reset the chat
    def reset():
        agent.reset()
        return [], update_markdown()
    
    # Function to save the current markdown
    def save():
        current_markdown = update_markdown()
        return save_markdown_as_docx(current_markdown)
    
    # Set up event handlers
    msg.submit(respond, [msg, chatbot], [msg,chatbot,markdown_display])
    submit_btn.click(respond, [msg, chatbot], [chatbot,markdown_display])
    reset_btn.click(reset, [], [chatbot, markdown_display])
    save_btn.click(save, [], [save_status])

# Launch the app
if __name__ == "__main__":
    demo.launch(debug=True) 

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
